# Aufgabe-G-WiederholteFragen

## Allgemeines

Eine allgemeine Beschreibung der Laboraufgaben inklusive des Vorgehens, den Bewertungsrichtlinien und der Abgabe finden Sie  <a href="ML-allgemein.ipynb">hier</a>

## Datenquelle


* Laden Sie ihre Daten von http://141.72.190.207/ml_lab/G_fragen herunter
    * Die Daten sind geschützt. 
        * Sie müssen evtl. in einem Netzwerk der DHBW (z.B. WLAN, VPN, ...) angemeldet sein. 
        * Sie können sich auf der Webseite mit dem Benutzernamen dhbw und dem Zugangsnamen ml_2021 anmelden. 
* Die Daten sind in einem anwendungsspezifischen Format gespeichert.
    * Sie finden evtl. Informationen über die Daten in einer "README" Datei. 
    * Finden Sie keine solche Datei sind die Daten selbst erklärend. 
    
    



## Aufgabe

Die Datensammlung enthält Fragen aus einem Forum. Es sind Paare aus Fragen gleichen Inhalts geboten.

Erstellen Sie ein tiefes Neuronales Netzwerk, dass dem Nutzer schon bei der Eingabe einer neuen Frage gleich ähnliche Fragen anbietet. 
* Je mehr Text der Benutzer eingibt, desto präziser sollen die ähnlichen Fragen sein! 




# Lösung

* Beginnen Sie hier mit Ihrer Dokumentation und Implementierung! 

In [3]:
# Fügen Sie ihren Code hier ein!

Vorgehen:
- Finden von unnötigen Wörtern (in, wie usw.)
- Extrahieren der "wertvollen" Informationswerten
- Einteilen der Fragen in Kategorien und Lernen KI anhand der gelieferten Paare
- Überprüfung anhand der Testfragen
- Erstellung einer Eingabemaske, mit Testfragen zur überprüfung der KI

# Einlesen der Daten

## Einlesen der Trainingsdaten

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers

train_data = pd.read_csv(
    "./fragen/train.csv",
    usecols= ['question1', 'question2', 'is_duplicate'],
    encoding='utf-8'
)

train_data.dropna(inplace=True)
train_data.head()
#for i in range(0, len(train_data)):
#    if type(train_data.at[i, 'question1']) == float or type(train_data.at[i, 'question2']) == float:
#        print(train_data.at[i, 'question2'])
#        print(train_data.at[i, 'question1'])
#        print(i)

,question1,question2,is_duplicate
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


,question1,question2,is_duplicate
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


## Einlesen der Testdaten

In [5]:
test_data = pd.read_csv(
    "./fragen/test.csv",
    index_col="test_id"
)

test_data.head()


/tmp/ipykernel_144957/1675913220.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  test_data = pd.read_csv(


,question1,question2
test_id,,
0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,What but is the best way to send money from Ch...,What you send money to China?
3,Which food not emulsifiers?,What foods fibre?
4,"How ""aberystwyth"" start reading?",How their can I start reading?


# Text preprocessing
According to the tensorflow website

In [6]:
target = train_data.pop('is_duplicate')

ds_raw = tf.data.Dataset.from_tensor_slices((train_data.values, target.values))

for ex in ds_raw.take(3):
    print(ex)

(<tf.Tensor: shape=(2,), dtype=string, numpy=
array([b'What is the step by step guide to invest in share market in india?',
       b'What is the step by step guide to invest in share market?'],
      dtype=object)>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(2,), dtype=string, numpy=
array([b'What is the story of Kohinoor (Koh-i-Noor) Diamond?',
       b'What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?'],
      dtype=object)>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(2,), dtype=string, numpy=
array([b'How can I increase the speed of my internet connection while using a VPN?',
       b'How can Internet speed be increased by hacking through DNS?'],
      dtype=object)>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)


2022-01-31 20:22:09.477307: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-01-31 20:22:09.477332: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-31 20:22:09.477347: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (maxhp14s): /proc/driver/nvidia/version does not exist
2022-01-31 20:22:09.477928: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Unterteilung in Trainings, Validierungs und Testdaten


In [7]:
tf.random.set_seed(1)

ds_length = len(ds_raw)
ds_raw_train = ds_raw.take(round(ds_length*0.8)-1)
ds_raw_train_valid = ds_raw.take(round(ds_length*0.2)-1)

## Create dictionary

In [8]:
from collections import Counter

tokenizer = tfds.deprecated.text.Tokenizer()
token_counts = Counter()

for example in ds_raw_train:
    for i in range(0,2):
        tokens = tokenizer.tokenize(example[0].numpy()[i])
        token_counts.update(tokens)

print("Größe des Wörterbuches: ", len(token_counts))

Größe des Wörterbuches:  99353


## Create a encoder and map the arrays

In [14]:
encoder = tfds.deprecated.text.TokenTextEncoder(token_counts)
encoder.encode('This is a example')

def encode(text_tensor, label):
    print(text_tensor)
    text0 = text_tensor.numpy()[0]
    text1 = text_tensor.numpy()[1]
    return encoder.encode(text0), encoder.encode(text1), label

def encode_map_fn(text, label):
    return tf.py_function(encode,
                          inp=[text, label],
                          Tout=[tf.int64, tf.int64, tf.int64])

ds_train = ds_raw_train.map(encode_map_fn)
ds_train_valid = ds_raw_train_valid.map(encode_map_fn)
for ex in ds_train.take(3):
    print(ex)

tf.Tensor(
[b'What is the step by step guide to invest in share market in india?'
 b'What is the step by step guide to invest in share market?'], shape=(2,), dtype=string)
(<tf.Tensor: shape=(14,), dtype=int64, numpy=array([ 1,  2,  3,  4,  5,  4,  6,  7,  8,  9, 10, 11,  9, 12])>, <tf.Tensor: shape=(12,), dtype=int64, numpy=array([ 1,  2,  3,  4,  5,  4,  6,  7,  8,  9, 10, 11])>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
tf.Tensor(
[b'What is the story of Kohinoor (Koh-i-Noor) Diamond?'
 b'What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?'], shape=(2,), dtype=string)
(<tf.Tensor: shape=(10,), dtype=int64, numpy=array([ 1,  2,  3, 13, 14, 15, 16, 17, 18, 19])>, <tf.Tensor: shape=(15,), dtype=int64, numpy=array([ 1, 20, 21, 22,  3, 23, 24, 25,  3, 15, 16, 17, 18, 26, 27])>, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
tf.Tensor(
[b'How can I increase the speed of my internet connection while using a VPN?'
 b'How can Internet speed be increased 